协程
---
协程，又称**微线程**，**纤程**。英文名**Coroutine**。

协程的概念很早就提出来了，但直到最近几年才在某些语言（如Lua）中得到广泛应用。

**子程序**，或者称为**函数**，在所有语言中都是**层级调用**，比如A调用B，B在执行过程中又调用了C，C执行完毕返回，B执行完毕返回，最后是A执行完毕。

所以子程序调用是通过栈实现的，一个线程就是执行一个子程序。

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同。

协程看上去也是子程序，但执行过程中，在子程序内部可**中断**，然后转而执行别的子程序，在适当的时候再返回来接着执行。

注意，在一个子程序中中断，去执行其他子程序，不是函数调用，有点类似CPU的中断。比如子程序A、B：

In [ ]:
def A():
    print('1')
    print('2')
    print('3')

def B():
    print('x')
    print('y')
    print('z')

假设由协程执行，在执行A的过程中，可以随时中断，去执行B，B也可能在执行过程中中断再去执行A，结果可能是：1 2 x y 3 z 

但是在A中是没有调用B的，所以协程的调用比函数调用理解起来要难一些。

看起来A、B的执行有点像多线程，但协程的特点在于是**一个线程执行**，那和多线程比，协程有何优势？

最大的优势就是协程**极高的执行效率**。因为子程序切换不是线程切换，而是由程序自身控制，因此，没有线程切换的开销，和多线程比，线程数量越多，协程的性能优势就越明显。

第二大优势就是**不需要多线程的锁机制**，因为只有一个线程，也不存在同时写变量冲突，在协程中控制共享资源不加锁，只需要判断状态就好了，所以执行效率比多线程高很多。

因为协程是一个线程执行，那怎么利用**多核CPU**呢？最简单的方法是**多进程+协程**，既充分利用多核，又充分发挥协程的高效率，可获得极高的性能。

Python对协程的支持是通过**generator**实现的。

在generator中，我们不但可以通过for循环来迭代，还可以不断调用next()函数获取由yield语句返回的下一个值。

但是Python的yield不但可以返回一个值，它还可以**接收调用者发出的参数**。

来看例子：

传统的**生产者-消费者**模型是一个线程写消息，一个线程取消息，通过锁机制控制队列和等待，但一不小心就可能死锁。

如果改用协程，生产者生产消息后，直接通过yield跳转到消费者开始执行，待消费者执行完毕后，切换回生产者继续生产，效率极高：

In [3]:
def consumer():
    r = ''
    while True:
        # 2.consumer通过yield拿到传递的None，yield跳出
        n = yield r
        # 4.从上次跳出的位置，接着往下执行
        if not n:
            return    # StopIteration
        print('[CONSUMER] Consuming %s ...' % n )
        r = '200 OK'
        # 6.从这里开始循环，到yield的时候，再跳出来

def produce(c):
    # 1.启动生成器，会跳到consumer
    c.send(None)
    # 3.接着往下执行，产生数据，通过c.send(n)，再切换到consumer
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s ...' % n )
        r = c.send(n)    # 切换到consumer执行
        # 7.跳出来后，函数返回值是200 OK，所以往下执行，print出200 OK
        print('[PRODUCER Consumer return: %s' % r)
        # 8.从这里开始循环前面的步骤，直到最后
    c.close()  # 关闭consumer

c = consumer()
produce(c)

[PRODUCER] Producing 1 ...
[CONSUMER] Consuming 1 ...
[PRODUCER Consumer return: 200 OK
[PRODUCER] Producing 2 ...
[CONSUMER] Consuming 2 ...
[PRODUCER Consumer return: 200 OK
[PRODUCER] Producing 3 ...
[CONSUMER] Consuming 3 ...
[PRODUCER Consumer return: 200 OK
[PRODUCER] Producing 4 ...
[CONSUMER] Consuming 4 ...
[PRODUCER Consumer return: 200 OK
[PRODUCER] Producing 5 ...
[CONSUMER] Consuming 5 ...
[PRODUCER Consumer return: 200 OK


注意到consumer函数是一个generator，把一个consumer传入produce后：

首先调用c.send(None)启动生成器；

然后，一旦生产了东西，通过c.send(n)切换到consumer执行；

consumer通过yield拿到消息，处理，又通过yield把结果传回；

produce拿到consumer处理的结果，继续生产下一条消息；

produce决定不生产了，通过c.close()关闭consumer，整个过程结束。

整个流程无锁，由一个线程执行，produce和consumer协作完成任务，所以称为“协程”，而非线程的抢占式多任务。

最后套用Donald Knuth的一句话总结协程的特点：

“子程序就是协程的一种特例。”

<br/>

yield可以返回值，yield语句本身也是有值的，可以通过send()函数的参数指定～

consumer()中的 n = yield r 语句，可以分开2步理解：
1. yiled r，即向调用方返回r
2. 赋值语句n = xxxx，等待调用方send()函数指定的yield r语句的值，将该值赋值给n

produce()中的 r = c.send(1)语句，这一步的操作可以分开2步理解：
1. 将生成器上一次挂起的yield语句的值指定为1，则consumer()中的n = xxxx就是n=1了;
2. 相当于执行r = next(c),consumer从上次挂起的yield语句处继续往下执行，再次执行到n = yield r就挂起，将r返回给produce()，n则等待调用方指定的yield r的值

参考：https://blog.csdn.net/hedan2013/article/details/56293173

生成器
---
对象是一个迭代器。但是它比迭代器对象多了一些方法，它们包括**send**方法，**throw**方法和**close**方法。这些方法，主要是用于**外部与生成器对象的交互**。

生成器对象的send方法
---
send方法有一个参数，该参数指定的是**上一次被挂起的yield语句的返回值**。这样说起来比较抽象，看下面的例子。

In [20]:
def MyGenerator():
    value = (yield 1)
    value = (yield value)
    value = (yield value)

gen = MyGenerator()
print(next(gen))
print(gen.send(5))
print(gen.send(8))

1
5
8


上面代码的运行过程如下。
当调用gen.next()方法时，python首先会执行MyGenerator方法的yield 1语句。由于是一个yield语句，因此方法的执行过程被挂起，而next方法返回值为yield关键字后面表达式的值，即为1。

当调用gen.send(5)方法时，python首先恢复MyGenerator方法的运行环境。同时，将表达式(yield 1)的返回值定义为send方法参数的值，即为5。这样，接下来value=（yield 1）这一赋值语句会将value的值置为5。继续运行会遇到yield value语句。因此，MyGenerator方法再次被挂起。同时，send方法的返回值为yield关键字后面表达式的值，也即value的值，为5。后面类似。

总的来说，send方法和next方法唯一的区别是在执行send方法会首先把上一次挂起的**yield语句的返回值通过参数设定**，从而实现与生成器方法的交互。但是需要注意，**在一个生成器对象没有执行next方法之前，由于没有yield语句被挂起，所以执行send方法会报错**。例如：

In [21]:
gen = MyGenerator()  
print(gen.send(2))

TypeError: can't send non-None value to a just-started generator

当然，下面的代码是可以接受的。  
因为当send方法的参数为None时，它与next方法完全等价。但是注意，虽然上面的代码可以接受，但是不规范。所以，**在调用send方法之前，还是先调用一次next方法为好**。

In [28]:
gen = MyGenerator()  
print(gen.send(None))

1


生成器对象的throw方法
---
它的实现手段是通过向生成器对象在**上次被挂起处，抛出一个异常**。之后会继续执行生成器对象中后面的语句，直至遇到下一个yield语句返回。如果在生成器对象方法执行完毕后，依然没有遇到yield语句，抛出StopIteration异常。

In [29]:
def myGenerator():
    value = 1
    while True:
        yield value
        value += 1


gen = myGenerator()
print(next(gen))
print(next(gen))
print(gen.throw(Exception, "Method throw called!"))

1
2


Exception: Method throw called!

代码的最后一句向生成器对象抛出了一个异常。但是，在生成器对象的方法时没有处理该异常的代码，因此异常会被抛出到主方法。

下面的示例中，添加了try-except语句块处理异常。当生成器方法收到异常后，会调到except语句块，将value置为1。

In [35]:
def myGenerator():
    value = 1
    while True:
        try:
            yield value
            value += 1
        except:
            value = 999


gen = myGenerator()
print(next(gen))
print(next(gen))
print(gen.throw(Exception, "Method throw called!"))

1
2
999


Exception ignored in: <generator object myGenerator at 0x111083e08>
RuntimeError: generator ignored GeneratorExit


上面输出中，999是gen.throw方法的返回值。在执行完该方法后，生成器对象方法的while循环并没有结束，也即是说生成器方法的执行还没有结束。这个时候如果强制结束主程序，会抛出一个RuntimeError。也就是上面输出的第4行。要优雅地关闭主程序，需要用到生成器对象的close方法。

生成器对象的close方法
---
生成器对象的close方法会在生成器对象方法的挂起处抛出一个GeneratorExit异常。GeneratorExit异常产生后，系统会继续把生成器对象方法后续的代码执行完毕。参见下面的代码。


In [42]:
def myGenerator():  
    try:
        yield 1
        print("Statement after yield")
    except GeneratorExit:
        print("Generator error caught")

    print("End of myGenerator")

gen = myGenerator()
print(next(gen))
gen.close()
print("End of main caller")


1
Generator error caught
End of myGenerator
End of main caller


代码执行过程如下：
1. 当调用gen.next方法时，会激活生成器，直至遇到生成器方法的yield语句，返回值1。同时，生成器方法的执行被挂起。
2. 当调用gen.close方法时，恢复生成器方法的执行过程。系统在yield语句处抛出GeneratorExit异常，执行过程跳到except语句块。当except语句块处理完毕后，系统会继续往下执行，直至生成器方法执行结束。

需要注意的是，GeneratorExit异常的产生意味着生成器对象的**生命周期已经结束**。因此，一旦产生了GeneratorExit异常，生成器方法后续执行的语句中，**不能再有yield语句**，否则会产生RuntimeError。请看下面的例子。


In [43]:
def myGenerator():  
    try:
        yield 1
        print("Statement after yield")
    except GeneratorExit:
        print("Generator error caught")

    yield 3

gen = myGenerator()
print(next(gen))
gen.close()
print("End of main caller")

1
Generator error caught


RuntimeError: generator ignored GeneratorExit

注意，由于RuntimError会向主方法抛出，因此主方法最后的print语句没有执行。

有了上面的知识，我们就可以理解为什么下面的代码会抛出RuntimError错误了。


In [64]:
def myGenerator():  
    value = 1  
    while True:  
        try:  
            yield value  
            value += 1  
        except:  
            value = 999  

gen = myGenerator()  
print(next(gen))
print(next(gen))
print(gen.throw(Exception, "Method throw called!"))  
print(next(gen))

1
2
999
1000


Exception ignored in: <generator object myGenerator at 0x11101a8e0>
RuntimeError: generator ignored GeneratorExit


上面代码中，当主程序结束前，系统产生GeneratorExit异常，被生成器对象方法的except语句捕获，但是此时while语句还没有退出，因此后面还会执行“yield value”这一语句，从而发生RuntimeError。要避免这个错误非常简单，请看下面的代码。

In [67]:
def myGenerator():  
    value = 1  
    while True:  
        try:  
            yield value  
            value += 1  
        except Exception:  
            value = 999  

gen = myGenerator()  
print(next(gen))
print(next(gen))
print(gen.throw(Exception, "Method throw called!"))  

1
2
999


代码第7行的except语句声明只捕获Exception异常对象。这样，当系统产生GeneratorExit异常后，不再被except语句捕获，继续向外抛出，从而跳出了生成器对象方法的while语句。

这里再简单说一句，GeneratorExit异常继承自BaseException类。BaseException类与Exception类不同。一般情况下，**BaseException**类是所有**内建异常类**的基类，而**Exception**类是所有**用户定义的异常类**的基类。